In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
# utils 
import math
import numpy as np
import cv2 as cv
import os

def image_none(img_path):
    image=cv.imread(img_path)
    return image

def show_image(image):
    cv.imshow('image', image) 
    cv.waitKey(0)
    cv.destroyAllWindows()

def euclidean_distance(node1, node2):
    return math.sqrt((node1[0] - node2[0])**2 + (node1[1] - node2[1])**2)

def manhattan_distance(node1, node2):
    return abs(node1[0] - node2[0]) + abs(node1[1] - node2[1])
  

def octile_distance(node1,node2):
    dx= node2[0] - node1[0]
    dy= node2[1] - node1[1]
    f = math.sqrt(2) - 1
    if dx < dy:
        return f * dx + dy
    else:
        return f * dy + dx

def coloring_pixel(image,color,pixel):
    if image is  not None:
        image[pixel[0],pixel[1]]=color
        return image

def define_start_goal(image,start,goal):
    start_color = [255, 0, 102] #pink 
    goal_color =  [0, 0, 204] #blue
    if image is not None:
        # should not be obstacle 
        if np.array_equal(image[start[0],start[1]],[229, 229, 229]) and np.array_equal(image[goal[0],goal[1]],[229, 229, 229]):
            image = coloring_pixel(image,start_color,start)
            image = coloring_pixel(image,goal_color,goal) 
            return image
        else: 
            print("Goal or Start is an obstacle, Redefine it!")
            return image

def draw_path(img_path,path):    
    image=cv.imread(img_path)
    path = set(path)
    if image is not None:
        for tuple in path:
            print(tuple)
            image[tuple[1],tuple[0]]=[0, 0, 255] #red
    return image

def save_image(img_path,path,image_name):
    image=cv.imread(img_path)
    new_image = draw_path(image,path)
    saving_path = '/content/'
    cv.imwrite(os.path.join(saving_path , image_name), new_image)    

def isValid(img_path,node):
    image=cv.imread(img_path,0)
    if image is not None:
        return  image[node[0],node[1]] == 229 



def imageToMatrix(img_path):
    image=cv.imread(img_path,0)
    rows,cols=(1073,1073)    
    # image (grey scale) to a matrix of 0 and 1 
    for i in range(rows):
        for j in range(cols):
            if image[i,j] == 0 or image[i,j] == 99: 
                image[i,j] = 0 #obstacle
            elif image[i,j] == 229:
                image[i,j] = 1
    return image                
    


In [ ]:
""" output structure: (room_name,method used, type de test, path, path length, operations, time)"""
class room_path:
    def __init__(self,room_name,method,test,path,path_length,operations,time):
        self.room_name=room_name
        self.method=method
        self.test=test
        self.path=path
        self.path_length=path_length
        self.operations=operations
        self.time=time
    def path_info(self):
        path_info = "room name : " + str(self.room_name)+", test : " + self.test +", path length : " + str(self.path_length) + ", time : " + str(self.time)
        return path_info
        
# sa 


# Imports 
import math
import random as rd 
import src.uav_path_planning.utils as ut
import cv2 as cv
import os


def findNeighbors(matrix,node):
    #check validity and return neighbors to choose randomly one of them
    neighbors=[]
    if matrix[node[0]+1,node[1]] == 1:
        neighbors.append((node[0]+1,node[1]))
    if matrix[node[0],node[1]+1]==1:
        neighbors.append((node[0],node[1]+1))

    if matrix[node[0]+1,node[1]+1]==1:
        neighbors.append((node[0]+1,node[1]+1))

    if matrix[node[0]-1,node[1]]==1:
        neighbors.append((node[0]-1,node[1]))
        
    if matrix[node[0],node[1]-1]==1:
        neighbors.append( (node[0],node[1]-1))

    if matrix[node[0]-1,node[1]-1]==1:
        neighbors.append((node[0]-1,node[1]-1))

    if matrix[node[0]-1,node[1]+1]==1:
        neighbors.append((node[0]-1,node[1]+1))

    if matrix[node[0]+1,node[1]-1]==1:
        neighbors.append((node[0]+1,node[1]-1)) 

    return neighbors

def generatePath(matrix,start,end): 
    path=[]
    path.append(start)
    node=start 
    while(node!=end):
        neighbors = findNeighbors(matrix,node)
        node=neighbors[rd.randint(0,len(neighbors)-2)]
        path.append(node)
    return path

def generateNeighboringPath(matrix,path): 
    x=rd.randint(0,len(path)-2)
    neighbors=findNeighbors(matrix,path[x])
    output=path.copy()
    output[x]=neighbors[rd.randint(0,len(neighbors)-2)] 
    output[x+1:]=generatePath(matrix,output[x+1],output[-1])
    return output

def cost(path):
    distance=0
    for i in range(len(path)-1):
        distance+=octile_distance(path[i],path[i+1])
    return distance

def sa(matrix,start,end , parameters):  
    CT = parameters['IT']  # Current temperature
    N= parameters['N']
    NT= parameters['NT'] 
    alpha = parameters['alpha']
    FT = parameters['FT']
    CP = generatePath(matrix,start,end) # current path (random)
    CL = cost(CP) 
    OP = CP # optimal path
    OL = CL
    for i in range(N):
        print("i= ",i)
        for j in range(NT):
            RP = generateNeighboringPath(matrix,CP)
            RL = cost(RP)  
            if RL < CL:
                CP = RP
                CL = RL 
            else: 
                p = math.exp(-(RL-CL)/CT) # Transition Probability 
                r= rd.uniform(0, 1)
                if r<p:
                    CP = RP
                    CL = RL 
            if CL < OL:
                OP = CP
                OL = CL
        CT *= alpha 
        if CT <= FT:
            break 
    return OP , OL 


# Save image
def SA(img_path,start,end,image_name,parameters,test):
    image=imageToMatrix(img_path)
    op , ol = sa(image,start,end,parameters) 
    #Draw path in the image and save it
    image=cv.imread(img_path)
    new_image = draw_path(img_path,op) 
    saving_path = '/content/'+test 
    cv.imwrite(os.path.join(saving_path ,image_name), new_image)
    return op, ol 


In [ ]:
""" 
for each image, in benchmark find path for the given start and end (save images with path) 
for each image, save time needed to look for the path
"""
from os import listdir
import time
import cv2 as cv


folder_dir = "/content/"
file = open("/content/all_results.txt","w") 


#-------------------PARAMETER TUNING---------------------------------------- 

file.write("\nPARAMETER TUNING \n")

IT = [ 1,2 ] 
FT = [ 0,1 ]
N =[ 1,2 ] 
NT = [ 1,2 ] 
alpha = [ 0.9 , 0.8 ]
images=["8room_000.png" ] #32 #,"16room_000.png","32room_000.png","64room_000.png"] # 128

start = (9, 9)
end = (10, 15)
easy_output =[]
i=0

for image in images:
    parameters = {}
    for it in IT:
        parameters['IT'] = it     # Initial temperature
        for ft in FT:
            parameters['FT'] = ft     # Final temperature 
            for n in N:
                parameters['N']  = n      # Number of iterations
                for nt in NT:
                    parameters['NT']= nt       # Number of iterations per Temperature 
                    for a in alpha:
                        parameters['alpha'] = a  # Geometric Coefficient alpha
                        img_path=folder_dir+"/"+str(image)
                        img=cv.imread(img_path,0)
                        if img is not None:
                            if isValid(img_path,start):
                                if isValid(img_path,end):
                                    print(str(image))
                                    start_time = time.time()
                                    path , length = SA(img_path,start,end,str(i)+"new_"+str(image),parameters,"easy")
                                    print("path = ", path)
                                    print("--- %s seconds ---" % (time.time() - start_time))
                                    easy_output.append(room_path(str(i)+"new_"+str(image),"SA","easy" ,path, length,0,time.time() - start_time))                                     
                                    pathInfo = easy_output[-1].path_info()
                                    print(pathInfo)
                                    temp= str(pathInfo) +"it: "+str(it)+" ft: "+str(ft)+" n: "+str(n)+" nt: "+str(nt)+" alpha: "+str(a) + "\n"
                                    print(temp)
                                    file.write(temp)
                                    i=i+1
                                else:
                                    print(str(image) ,"invalid end")
                            else:
                                print(str(image),"invalid start")
                        else:
                            print("image is None")
"""
parameters={'IT':1,'FT':1,'N':1,'NT':1,'alpha':0.9}

#-------------------EASY TEST---------------------------------------- 

start = (133, 30)
end = (21, 201)

easy_output=[]
for image in listdir(folder_dir):
    img_path=folder_dir+"/"+str(image)
    img=cv.imread(img_path,0)
    if img is not None:
        if isValid(img_path,start):
            if isValid(img_path,end):
                #print(str(image))
                start_time = time.time()
                path , length = SA(img_path,start,end,"new_"+str(image),parameters,"easy")
                #print("--- %s seconds ---" % (time.time() - start_time))
                easy_output.append(room_path.room_path("new_"+str(image),"SA","easy",path,0 , length,time.time() - start_time))
            else:
                print(str(image) ,"invalid end")
        else:
            print(str(image),"invalid start")
    else:
        print("image is None")


file.write("\nEASY TEST \n")
print("length (medium output) : ",len(easy_output))        
for i in range(len(easy_output)):
    pathInfo = easy_output[i].path_info()
    print(pathInfo)
    file.write((pathInfo+"\n"))


#-------------------Medium TEST---------------------------------------- 

start = (18, 30)
end = (21, 1001)

medium_output=[]
for image in listdir(folder_dir):
    img_path=folder_dir+"/"+str(image)
    img=cv.imread(img_path,0)
    if img is not None:
        if isValid(img_path,start):
            if isValid(img_path,end):
                #print(str(image))
                start_time = time.time()
                path , length = SA(img_path,start,end,"new_"+str(image),parameters,"medium")
                #print("--- %s seconds ---" % (time.time() - start_time))
                medium_output.append(room_path.room_path("new_"+str(image),"SA","medium",path,0 , length,time.time() - start_time))
            else:
                print(str(image) ,"invalid end")
        else:
            print(str(image),"invalid start")
    else:
        print("image is None")

# CHANGE START , END

file.write("\nMEDIUM TEST \n")
print("length (medium output) : ",len(medium_output))        
for i in range(len(medium_output)):
    pathInfo = medium_output[i].path_info()
    print(pathInfo)
    file.write((pathInfo+"\n"))


#-------------------HARD TEST---------------------------------------- 


start = (12, 30)
end = (1040, 990)

hard_output=[]
for image in listdir(folder_dir):
    img_path=folder_dir+"/"+str(image)
    img=cv.imread(img_path,0)
    if img is not None:
        if isValid(img_path,start):
            if isValid(img_path,end):
                #print(str(image))
                start_time = time.time()
                path , length = SA(img_path,start,end,"new_"+str(image),parameters,"hard")
                #print("--- %s seconds ---" % (time.time() - start_time))
                hard_output.append(room_path.room_path("new_"+str(image),"SA","hard",path ,0, length,time.time() - start_time))
            else:
                print(str(image) ,"invalid end")
        else:
            print(str(image),"invalid start")
    else:
        print("image is None")

file.write("\nHARD TEST \n")
print("length (hard output) : ",len(hard_output))        
for i in range(len(hard_output)):
    pathInfo = hard_output[i].path_info()
    print(pathInfo)
    file.write((pathInfo+"\n"))
"""